In [2]:
%pip install transformers
%pip install datasets
%pip install peft
%pip install bitsandbytes
%pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
import torch
from tqdm import tqdm
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
from transformers import pipeline
from transformers import BitsAndBytesConfig
import sentencepiece

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/CS\ 182\ Final\ Project\/Phase\ 2

/content/drive/.shortcut-targets-by-id/1hzhdcGA40OipfzF0SRT7omKvzCSi0q4r/CS 182 Final Project/Phase 2


In [6]:
from datasets import Dataset, load_dataset

In [7]:
dataset = load_dataset('HarshaniMadhushaniBandara/llma_math_word_problem') ## Fill in Dataset
dataset = dataset['train'].select_columns(["text"]) # select appropriate column names from dataset
dataset = dataset.train_test_split(test_size=0.2, seed=20)
train, val = dataset['train'], dataset['test']

val_humaneval = val.train_test_split(test_size=100, seed=20)['test']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
for example in tqdm(val_humaneval):
  result = example["text"].split("[/INST]")
  print(result[0] + " [/INST]")
  print(result[1])
  break

  0%|          | 0/100 [00:00<?, ?it/s]

"<s> [INST] Considering math constraints and units, using Sinhala language, Create a math word problem starting with සැමීට පැය 3කින් තණකොළ කැපීමට  [/INST]
  සැමීට පැය 3කින් තණකොළ කැපීමට හැකි අතර ලෝරාට පැය 4කින් එය කළ හැකි නම්, ඔවුන් එකට වැඩ කළහොත් කොපමණ කාලයක් ගතවේද? </s>"


In [ ]:
def get_test_ppl(model, tokenizer, dataset, base_model=False, device='cuda'):

  nlls = []

  for example in tqdm(dataset):

    q_encoding_length = tokenizer(q, return_tensors="pt").input_ids.shape[1]
    qa_encoding = tokenizer(qa, return_tensors='pt')
    input_ids = qa_encoding.input_ids
    target_ids = input_ids.clone()
    target_ids[:, :q_encoding_length] = -100

    with torch.no_grad():
      output = model(input_ids.to(device), labels=target_ids.to(device))
      nlls.append(output.loss.item())

  df = pd.DataFrame(dataset)
  df['nll'] = nlls
  df['ppl'] = torch.exp(torch.tensor(nlls))
  df = df[['prompt', 'question', 'nll', 'ppl']]
  df.to_csv(f'./problem_writing_results/human_eval.csv', index=False)
  return df

### Experiment-specific arguments

In [ ]:
model_name = ["gpt2-xl_problem_solving"]

runs = {}

for final in final_gpt_models:
  runs[final + "_final"] = prefix + final



In [1]:
device = "cuda"
base_model = True
base_model_id = "gpt2-xl"

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

NameError: ignored

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("gpt2-xl_problem_solving").to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1
df = get_test_ppl(model, tokenizer, val_humaneval, base_model, device)
print(f"{run}: {df['ppl'].mean()}")
del model

100%|██████████| 409/409 [00:49<00:00,  8.19it/s]


gpt2-medium_full_final: 1195489099776.0


KeyboardInterrupt: ignored

In [ ]:
# base model
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2").to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1
df = get_test_ppl(model, tokenizer, test_dataset, "llama_base", base_model=True, device=device)
print(f"llama_base: {df['ppl'].mean()}")

100%|██████████| 409/409 [00:50<00:00,  8.10it/s]

llama_base: 4.78560209274292


### Generate responses

In [ ]:
humaneval_df = read_files("Data Processing/Human Eval Data", ["126.csv", "127.csv", "182.csv", "189_fa.csv", "189_sp.csv"])
inputs = humaneval_df['question'].apply(lambda x: f"<s>[INST] {x} [/INST] ").to_list()
humaneval_df

,question,answer,class
0,Explain why entropy is always a non-negative q...,Entropy is non-negative because it is defined ...,126
1,Explain the Gram-Schmidt process and how it is...,The Gram-Schmidt process is an algorithm used ...,126
2,How do you compute the expected time until all...,To compute the expected time until all 20 ligh...,126
3,How is the joint probability mass function (PM...,"For discrete random variables $X$ and $Y$, the...",126
4,Summarize the relationship between reversibili...,"Within the context of Markov chains, a reversi...",126
...,...,...,...
6,What is the relationship between the primal an...,"In Kernel Ridge Regression, the primal weight ...",189_sp
7,Explain the concept of Kernel Perceptrons and ...,"In a Kernel Perceptron algorithm, the weights ...",189_sp
8,Explain the concept of using nonlinearity in n...,Nonlinearity in neural networks is introduced ...,189_sp
9,Explain why the naive computation of gradients...,The naive computation of gradients is ineffici...,189_sp


In [ ]:
# fine-tuned model
model = AutoPeftModelForCausalLM.from_pretrained("Model/mir/llama_lora_r64_a64/results/checkpoint-350").to(device)
pipe_tuned = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device=device)
results_tuned = pipe_tuned(inputs)
results_tuned = [result[0]['generated_text'] for result in results_tuned]
humaneval_df_tuned = humaneval_df.copy()
humaneval_df_tuned['output'] = results_tuned
humaneval_df_tuned.to_csv("Data Processing/Human Eval Data/results/llama_lora_r64_a64.csv", index=False)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [ ]:
# base model
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2").to(device)
pipe_base = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device=device)
results_base = pipe_base(inputs)
results_base = [result[0]['generated_text'] for result in results_base]
humaneval_df_base = humaneval_df.copy()
humaneval_df_base['output'] = results_base
humaneval_df_base.to_csv("Data Processing/Human Eval Data/results/llama_base.csv", index=False)